In [1]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.4 MB/s eta 0:00:00


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from torch import cuda
import torch
from transformers import DistilBertConfig

# Define the label mapping
label_mapping = {
    'sexist': 1,
    'not sexist': 0
}

category_mapping = {
    'none': 0,
    '1. threats, plans to harm and incitement': 1,
    '2. derogation': 2,
    '3. animosity': 3,
    '4. prejudiced discussions': 4
    # Add more categories as necessary
}

vector_mapping = {
    'none': 0,
    '1.1 threats of harm': 1,
    '1.2 incitement and encouragement of harm': 2,
    '2.1 descriptive attacks': 3,
    '2.2 aggressive and emotive attacks': 4,
    '2.3 dehumanising attacks & overt sexual objectification': 5,
    '3.1 casual use of gendered slurs, profanities, and insults': 6,
    '3.2 immutable gender differences and gender stereotypes': 7,
    '3.3 backhanded gendered compliments': 8,
    '3.4 condescending explanations or unwelcome advice': 9,
    '4.1 supporting mistreatment of individual women': 10,
    '4.2 supporting systemic discrimination against women as a group': 11
    # Add more vectors as necessary
}

# Step 1: Preprocess the data

# Load and preprocess the data
data = pd.read_csv('/content/edos_labelled_aggregated.csv')
labels = data['label_sexist'].values
categories = data['label_category'].values
vectors = data['label_vector'].values
texts = data['text'].values

train_texts, test_texts, train_labels, test_labels, train_categories, test_categories, train_vectors, test_vectors = train_test_split(
    texts, labels, categories, vectors, test_size=0.2, random_state=42
)

#train_texts, val_texts, train_labels, val_labels, train_categories, val_categories, train_vectors, val_vectors = train_test_split(
#    train_texts, train_labels, train_categories, train_vectors, test_size=0.25, random_state=42
#)

# Assuming you have your data stored in lists: texts and labels
#texts = [...]  # List of input texts
#labels = [...]  # List of corresponding labels

# Map labels to numerical values
#labels = [label_mapping[label] for label in labels]

# Map category values to numerical values
#categories = [...]  # List of category values
#categories = [category_mapping[category] for category in categories]

# Map vector values to numerical values
#vectors = [...]  # List of vector values
#vectors = [vector_mapping[vector] for vector in vectors]

# Step 2: Prepare the data for training

# Split the data into training and validation sets
#train_texts = texts[:train_size]
#train_categories = categories[:train_size]
#train_vectors = vectors[:train_size]
#train_labels = labels[:train_size]

#val_texts = texts[train_size:]
#val_categories = categories[train_size:]
#val_vectors = vectors[train_size:]
#val_labels = labels[train_size:]

# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# Convert train_texts and val_texts to the correct format
train_texts = [str(text) for text in train_texts]
test_texts = [str(text) for text in test_texts]
# Tokenize the text
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)



In [4]:
# Convert tokenized text into input features
class OnlineSexismDataset(Dataset):
  def __init__(self, texts, labels, categories, vectors, tokenizer, max_len):
          self.texts = texts
          self.labels = labels
          self.categories = categories
          self.vectors = vectors
          self.tokenizer = tokenizer
          self.max_len = max_len
    
  def __len__(self):
        return len(self.texts)
    
  def __getitem__(self, idx):
      text = str(self.texts[idx])
      label = self.labels[idx]
      category = self.categories[idx]
      vector = self.vectors[idx]
    
      encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          truncation=True,
          max_length=self.max_len,
          padding='max_length',
          return_tensors='pt'
      )
    
      return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label_mapping[label]),  # Encode label as numerical value
            'category': torch.tensor(category_mapping[category]),  # Encode category as numerical value
            'vector': torch.tensor(vector_mapping[vector])
        }

train_dataset = OnlineSexismDataset(train_texts, train_labels, train_categories, train_vectors, tokenizer, 128)
test_dataset = OnlineSexismDataset(test_texts, test_labels, test_categories, test_vectors, tokenizer, 128)



In [5]:
class CustomRobertaForSequenceClassification(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size + 1 + 1 + 1 , config.num_labels)  # Include 1 additional unit for each extra feature
    
    def forward(self, input_ids=None, attention_mask=None, categories=None, labels=None, vectors=None, **kwargs):
        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        hidden_state = roberta_output.last_hidden_state[:, 0, :]  # Extract the [CLS] token embedding
        hidden_state = self.dropout(hidden_state)
    
        # Reshape the category tensor to match the dimensions of the hidden_state tensor
        if categories is not None:
            categories = categories.unsqueeze(1)  # Add an extra dimension
        
        # Reshape the labels tensor to match the dimensions of the hidden_state tensor
        if labels is not None:
            labels = labels.unsqueeze(1)  # Add an extra dimension
    
        # Reshape the vector tensor to match the dimensions of the hidden_state tensor
        if vectors is not None:
            vectors = vectors.unsqueeze(1)  # Add an extra dimension
    
        # Concatenate the hidden state with the extra features
        if categories is not None:
            hidden_state = torch.cat((hidden_state, categories), dim=1)
        if labels is not None:
            hidden_state = torch.cat((hidden_state, labels), dim=1)
        if vectors is not None:
            hidden_state = torch.cat((hidden_state, vectors), dim=1)
    
        logits = self.classifier(hidden_state)
        outputs = (logits,) + roberta_output[1:]  # Add hidden states and attention if they are present
    
        return outputs


In [11]:
# Step 3: Define and train the sequential classifier
from transformers import RobertaModel
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 5
criterion = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    test_loss = 0
    predictions = []
    true_labels = []
    true_categories = []
    true_vectors = []
    predicted_categories = []
    predicted_labels = []
    category_scores = {}  # Dictionary to store accuracy for each category
    label_scores = {}
    vector_scores = {}

    for batch in train_loader:
        

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        categories = batch['category'].to(device)
        vectors = batch['vector'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, categories=categories, vectors=vectors, labels=labels)
        logits = outputs[0]
        loss = criterion(logits, labels)  # Calculate the loss
        
               
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_train_loss:.4f}')

    # Validation
    model.eval()
    #test_predictions = []
    #test_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            categories = batch['category'].to(device)
            vectors = batch['vector'].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                categories=categories,
                vectors=vectors
            )
            
            logits = outputs[0]
            loss = criterion(logits, labels)
            
            #optimizer.step()
            test_loss += loss.item()
            
            logits = outputs[0]
            softmax_probs = torch.softmax(logits, dim=1)
            predicted_labels_batch = torch.argmax(softmax_probs, dim=1)
        
            predictions.extend(predicted_labels_batch.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            
            predicted_labels.extend(predicted_labels_batch.cpu().numpy())
            true_vectors.extend(vectors.cpu().numpy())

            true_categories.extend(categories.cpu().numpy())
            predicted_categories.extend(predicted_labels_batch.cpu().numpy())
            
            
            
            # Calculate precision, recall, and F1-score for each label
            for i in range(len(predicted_labels_batch)):
                true_label = labels[i].item()
                predicted_label = predicted_labels_batch[i].item()
                sexist_label = test_labels[i]
            
                if sexist_label not in label_scores:
                    label_scores[sexist_label] = {'true_positive': 0, 'false_positive': 0, 'false_negative': 0, 'true_negative': 0}
            
                if true_label == predicted_label == 1:
                    label_scores[sexist_label]['true_positive'] += 1
                elif true_label == 0 and predicted_label == 1:
                    label_scores[sexist_label]['false_positive'] += 1
                elif true_label == 1 and predicted_label == 0:
                    label_scores[sexist_label]['false_negative'] += 1
                elif true_label == 0 and predicted_label == 0:
                    label_scores[sexist_label]['true_negative'] += 1
         
    for sexist_label, scores in label_scores.items():
        true_positive = scores['true_positive']
        false_positive = scores['false_positive']
        false_negative = scores['false_negative']
        true_negative = scores['true_negative']
    
        label_accuracy = (true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative)
        label_precision = true_positive / (true_positive + false_positive + 1e-10)
        label_recall = true_positive / (true_positive + false_negative + 1e-10)
        label_f1 = 2 * (label_precision * label_recall) / (label_precision + label_recall + 1e-10)
    
    print('Overall acore')
    print(f'Accuracy: {label_accuracy:.4f}')
    print(f'Precision: {label_precision:.4f}')
    print(f'Recall: {label_recall:.4f}')
    print(f'F1-score: {label_f1:.4f}')
    print('---')

    avg_test_loss = test_loss / len(test_loader)
      
    # Calculate cumulative scores
    print(f'Test Loss: {avg_test_loss}')

    
    # Calculate cumulative scores for vectors, labels, and categories
    #cumulative_true_positive = sum([score['true_positive'] for score in category_scores.values()])
    #cumulative_false_positive = sum([score['false_positive'] for score in category_scores.values()])
    #cumulative_false_negative = sum([score['false_negative'] for score in category_scores.values()])

    #cumulative_precision = cumulative_true_positive / (cumulative_true_positive + cumulative_false_positive + 1e-7)
    #cumulative_recall = cumulative_true_positive / (cumulative_true_positive + cumulative_false_negative + 1e-7)
    #cumulative_accuracy = (cumulative_true_positive + len(test_loader) - cumulative_false_positive - cumulative_false_negative) / len(test_loader)

    vector_accuracy = accuracy_score(true_vectors, predicted_labels)
    vector_precision = precision_score(true_vectors, predicted_labels, average='micro')
    vector_recall = recall_score(true_vectors, predicted_labels, average='micro')
    vector_f = f1_score(true_vectors, predicted_labels, average='micro')

    label_accuracy = accuracy_score(true_labels, predictions)
    label_precision = precision_score(true_labels, predictions, average='micro')
    label_recall = recall_score(true_labels, predictions, average='micro')
    label_f = f1_score(true_labels, predictions, average='micro')

    category_accuracy = accuracy_score(true_categories, predicted_categories)
    category_precision = precision_score(true_categories, predicted_categories, average='micro')
    category_recall = recall_score(true_categories, predicted_categories, average='micro')
    category_f = f1_score(true_categories, predicted_categories, average='micro')

   

    print("\nScores for Vectors:")
    print(f"Accuracy: {vector_accuracy:.4f}")
    print(f"Precision: {vector_precision:.4f}")
    print(f"Recall: {vector_recall:.4f}")
    print(f"F1: {vector_f:.4f}")

    print("\nScores for Labels:")
    print(f"Accuracy: {label_accuracy:.4f}")
    print(f"Precision: {label_precision:.4f}")
    print(f"Recall: {label_recall:.4f}")
    print(f"F1: {label_f:.4f}")

    print("\nScores for Categories:")
    print(f"Accuracy: {category_accuracy:.4f}")
    print(f"Precision: {category_precision:.4f}")
    print(f"Recall: {category_recall:.4f}")
    print(f"F1: {category_f:.4f}")

Some weights of the model checkpoint at roberta-base were not used when initializing CustomRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this mo

Epoch 1/5, Average Loss: 0.4113
Overall acore
Accuracy: 0.8520
Precision: 0.8208
Recall: 0.5239
F1-score: 0.6396
---
Test Loss: 0.33723324674367905

Scores for Vectors:
Accuracy: 0.7330
Precision: 0.7330
Recall: 0.7330
F1: 0.7330

Scores for Labels:
Accuracy: 0.8538
Precision: 0.8538
Recall: 0.8538
F1: 0.8537

Scores for Categories:
Accuracy: 0.7370
Precision: 0.7370
Recall: 0.7370
F1: 0.7370
Epoch 2/5, Average Loss: 0.2714
Overall acore
Accuracy: 0.8580
Precision: 0.8075
Recall: 0.5691
F1-score: 0.6677
---
Test Loss: 0.33886235021054745

Scores for Vectors:
Accuracy: 0.7335
Precision: 0.7335
Recall: 0.7335
F1: 0.7335

Scores for Labels:
Accuracy: 0.8718
Precision: 0.8718
Recall: 0.8718
F1: 0.8718

Scores for Categories:
Accuracy: 0.7392
Precision: 0.7392
Recall: 0.7392
F1: 0.7392
Epoch 3/5, Average Loss: 0.1903
Overall acore
Accuracy: 0.8740
Precision: 0.7619
Recall: 0.7234
F1-score: 0.7422
---
Test Loss: 0.318281247779727

Scores for Vectors:
Accuracy: 0.7063
Precision: 0.7063
Recall